<div align="center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</div>

<h1 align="center">Regressiong Model using Deep Learning with Keras</h1>


### Importing libraries

In [169]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

## Part A. Building a baseline model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the **adam** optimizer and the **mean squared error**  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing.

2. Train the model on the training data using **50 epochs**.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

4. Repeat steps 1 - 3, **50 times**, i.e., create a list of 50 mean squared errors.

5. Report the **mean and the standard deviation of the mean squared errors**.

### Load data

In [170]:
data = pd.read_csv('https://cocl.us/concrete_data')
data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


### Check data for missing values

In [171]:
data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

### Splitting data into predictor and target dataframes

In [172]:
predictors = data[data.columns[data.columns != 'Strength']]
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [173]:
target = data['Strength']
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Define Regressiong Model

In [174]:
def reg_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors.shape[1],)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### Train and Test the Network

Here, the train/test cycled is repeated 50 times:

In [186]:
errors = np.zeros(50)
model = reg_model()
for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    model.fit(x_train, y_train, epochs=50, verbose=0)
    y_pred = model.predict(x_test)
    errors[i] = mean_squared_error(np.asarray(y_test,dtype='float32'), np.asarray(y_pred, dtype='float32'))

### Reporting Mean and Standard Deviation of Mean Squared Errors

The errors are converted to a Pandas data frame. Mean and standard deviation are reported via the describe() method.

In [187]:
MSEDF = pd.DataFrame(errors, columns=['Mean Squared Errors'])
MSEDF.describe()

,Mean Squared Errors
count,50.000000
mean,80.543803
std,53.091670
min,43.748959
25%,51.097242
50%,54.776535
75%,108.763945
max,363.219421


## Part B. Normalise the data

Repeat Part A but **use a normalised version of the data**. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

Normalisation is done via sklearn.preprocessing.StandardScaer()

In [184]:
errors_norm = np.zeros(50)
model = reg_model()
for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    x_train_norm = StandardScaler().fit_transform(x_train) # Training data normalisation
    x_test_norm = StandardScaler().fit_transform(x_test) # Testing data normalisation
    model.fit(x_train_norm, y_train, epochs=50, verbose=0)
    y_pred = model.predict(x_test_norm)
    errors_norm[i] = mean_squared_error(np.asarray(y_test,dtype='float32'), np.asarray(y_pred, dtype='float32'))

In [185]:
MSEDF_norm = pd.DataFrame(errors_norm, columns=['Mean Squared Errors'])
MSEDF_norm.describe()

,Mean Squared Errors
count,50.000000
mean,50.974317
std,54.696231
min,26.896503
25%,33.241408
50%,37.995947
75%,44.541113
max,390.267365


### How does the mean of the mean squared errors compare to that from Step A?
By normalising the data, the average MSE decreased significantly. Data normalisation is therefore a desirable step to undertake prior to model fitting.

## Part C. Increase the number of epochs

Repeat Part B but use **100 epochs** this time for training.

In [179]:
errors_norm_100epch = np.zeros(50)
model = reg_model()
for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    x_train_norm = StandardScaler().fit_transform(x_train)
    x_test_norm = StandardScaler().fit_transform(x_test)
    model.fit(x_train_norm, y_train, epochs=100, verbose=0) # Number of epochs increased to 100
    y_pred = model.predict(x_test_norm)
    errors_norm_100epch[i] = mean_squared_error(np.asarray(y_test,dtype='float32'), np.asarray(y_pred, dtype='float32'))

In [180]:
MSEDF_norm_100epch = pd.DataFrame(errors_norm_100epch, columns=['Mean Squared Errors'])
MSEDF_norm_100epch.describe()

,Mean Squared Errors
count,50.000000
mean,47.537832
std,22.768403
min,33.255379
25%,39.188964
50%,42.276796
75%,47.908565
max,188.991272


### How does the mean of the mean squared errors compare to that from Step B?
After increasing the number of epochs from 50 to 100, mean MSE decreased slightly, while MSE standard deviation decreased 2-fold.

## Part D. Increase the number of hidden layers
Repeat part B but use a neural network with the following instead:
- Three hidden layers, each of 10 nodes and ReLU activation function.



In [181]:
def deeper_reg_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(predictors.shape[1],)))
    model.add(Dense(10, activation='relu', input_shape=(predictors.shape[1],)))
    model.add(Dense(10, activation='relu', input_shape=(predictors.shape[1],)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [182]:
errors_norm_deeper = np.zeros(50)
model = deeper_reg_model()
for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    x_train_norm = StandardScaler().fit_transform(x_train) # Training data normalisation
    x_test_norm = StandardScaler().fit_transform(x_test) # Testing data normalisation
    model.fit(x_train_norm, y_train, epochs=50, verbose=0)
    y_pred = model.predict(x_test_norm)
    errors_norm_deeper[i] = mean_squared_error(np.asarray(y_test,dtype='float32'), np.asarray(y_pred, dtype='float32'))

In [183]:
MSEDF_norm_deeper = pd.DataFrame(errors_norm_deeper, columns=['Mean Squared Errors'])
MSEDF_norm_deeper.describe()

,Mean Squared Errors
count,50.000000
mean,38.678722
std,13.865527
min,25.830202
25%,31.474419
50%,35.807642
75%,42.118161
max,118.480858


### How does the mean of the mean squared errors compare to that from Step B?

Both the mean and standard deviation of MSEs have decreased, although not by a lot. However in this case it seems that a deeper neural network contributes to smaller model errors.